In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.1871
Epoch [1/5], Step [200/600], Loss: 2.1006
Epoch [1/5], Step [300/600], Loss: 1.9995
Epoch [1/5], Step [400/600], Loss: 1.8989
Epoch [1/5], Step [500/600], Loss: 1.8588
Epoch [1/5], Step [600/600], Loss: 1.7859
Epoch [2/5], Step [100/600], Loss: 1.6895
Epoch [2/5], Step [200/600], Loss: 1.6525
Epoch [2/5], Step [300/600], Loss: 1.6888
Epoch [2/5], Step [400/600], Loss: 1.5895
Epoch [2/5], Step [500/600], Loss: 1.5615
Epoch [2/5], Step [600/600], Loss: 1.4472
Epoch [3/5], Step [100/600], Loss: 1.4368
Epoch [3/5], Step [200/600], Loss: 1.3545
Epoch [3/5], Step [300/600], Loss: 1.3867
Epoch [3/5], Step [400/600], Loss: 1.2446
Epoch [3/5], Step [500/600], Loss: 1.2437
Epoch [3/5], Step [600/600], Loss: 1.1830
Epoch [4/5], Step [100/600], Loss: 1.1716
Epoch [4/5], Step [200/600], Loss: 1.2986
Epoch [4/5], Step [300/600], Loss: 1.2028
Epoch [4/5], Step [400/600], Loss: 1.1677
Epoch [4/5], Step [500/600], Loss: 1.0295
Epoch [4/5], Step [600/600], Loss:

In [6]:
total_step, images.shape, labels.shape

(600, torch.Size([100, 784]), torch.Size([100]))

In [7]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the model on the 10000 test images: 82 %


In [9]:
images.shape, labels.shape, predicted.shape

(torch.Size([100, 784]), torch.Size([100]), torch.Size([100]))

In [10]:
model.state_dict()

OrderedDict([('weight',
              tensor([[ 0.0349, -0.0331,  0.0314,  ..., -0.0334,  0.0339, -0.0235],
                      [ 0.0223, -0.0080,  0.0079,  ..., -0.0355, -0.0285,  0.0265],
                      [ 0.0026, -0.0271,  0.0303,  ..., -0.0134,  0.0176, -0.0277],
                      ...,
                      [ 0.0143, -0.0224,  0.0250,  ..., -0.0290, -0.0024, -0.0306],
                      [-0.0136,  0.0333,  0.0003,  ..., -0.0091,  0.0110, -0.0152],
                      [-0.0331,  0.0323,  0.0115,  ...,  0.0251,  0.0183, -0.0293]])),
             ('bias', tensor(1.00000e-02 *
                     [-1.8764,  2.3952,  0.4064, -1.1929, -0.5676,  3.1836, -0.0506,
                      -0.4835, -6.5613,  1.4410]))])

In [12]:
model.state_dict()['weight'].shape, model.state_dict()['bias'].shape

(torch.Size([10, 784]), torch.Size([10]))